In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import math

# termination criteria
# cv2.TERM_CRITERIA_EPS : 정해둔 오차범위내에 다다르면, 알고리즘을 멈춘다.
# cv2.TERM_CRITERIA_MAX_ITER : 지정한 횟수에 다다르면, 알고리즘을 멈춘다.
#(알고리즘 제동조건, 횟수, 오차범위)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# object point들을 미리 설정

#objp = np.zeros((체스판 가로, 체스판 세로)) 배열 선언 및 초기화
objp = np.zeros((4*6,3), np.float32)

#objp의 모든행의 [0:2]인덱스 부분에 번호를 부여
objp[:,:2] = np.mgrid[0:6,0:4].T.reshape(-1,2)

# 모든 이미지에서 객체 포인트와 이미지 포인트를 저장하는 배열
objpoints = [] # 실제 세상에서의 3d 포인트들을 넣을 배열
imgpoints = [] # image plane에서의 2d 포인트들을 넣을 배열

images = glob.glob('./*.jpg')

for fname in images:
    img = cv2.imread(fname)
    
    #흑백으로 변경
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # 체스보드 코너들을 찾는 부분
    ret, corners = cv2.findChessboardCorners(gray, (6,4),None)
    
    # 만약, 찾았으면, objpoints, imgpoints에 추가
    if ret == True:
        objpoints.append(objp)
        
        #입력 영상 image에서 검출된 코너점 corners를 입력하여 코너점의 위치를 부화소 수준으로 다시 계산하여 반환한다.
        
        #cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        
        # 검출한 코너들을 보여주는 부분, 현재, 응답오류로 인해서 제외시켰다.
        img = cv2.drawChessboardCorners(img, (6,4), corners2,ret)
        
        img = cv2.resize(img, dsize=(640, 480), interpolation=cv2.INTER_AREA)

        #cv2.imshow('img', img)
        #cv2.waitKey(500)

#cv2.destroyAllWindows()

In [2]:
# 내부 파라미터 : camera calibration 결과, cameramatrix, 왜곡계수, 회전벡터, 병진벡터
_, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [3]:
# 외부 파라미터 : pose estimation 결과, 회전벡터, 병진벡터
_, rvec, tvec = cv2.solvePnP(objp, corners2, mtx, dist)

In [4]:
# 회전벡터를 rotation Matrix 폼으로 변경
rotM = cv2.Rodrigues(rvec)[0]

In [5]:
print(rotM)

[[ 0.01434366  0.99981083  0.01313602]
 [-0.9964932   0.01321051  0.08262435]
 [ 0.08243519 -0.01427509  0.99649419]]


In [6]:
print(tvec)

[[-1.78668095]
 [ 0.99838461]
 [ 3.53263705]]


In [7]:
#4x4 Matrix형태로 변환
A = np.hstack([rotM, tvec])
A = np.vstack([A, [0, 0, 0, 1]])

In [8]:
print(A)

[[ 0.01434366  0.99981083  0.01313602 -1.78668095]
 [-0.9964932   0.01321051  0.08262435  0.99838461]
 [ 0.08243519 -0.01427509  0.99649419  3.53263705]
 [ 0.          0.          0.          1.        ]]


In [9]:
#로봇 파라미터부분
B = np.zeros((4, 4))

X = float(input("Base to TCP Px값을 입력하시오."))
Y = float(input("Base to TCP Py값을 입력하시오."))
Z = float(input("Base ti TCP Pz값을 입력하시오."))

R = input("각도는 라디안 값입니까? Y/N")

if((R == "Y") or (R == "y")):
    yaw = float(input("Robot Parameter의 Rx(yaw)값을 입력하시오."))
    pitch = float(input("Robot Parameter의 Ry(pitch)값을 입력하시오."))
    roll = float(input("Robot Parameter의 Rz(Roll)값을 입력하시오."))
    
    yaw = math.degrees(yaw)
    pitch = math.degrees(pitch)
    roll = math.degrees(roll)

else:
    yaw = float(input("Robot Parameter의 Rx(yaw)값을 입력하시오."))
    pitch = float(input("Robot Parameter의 Ry(pitch)값을 입력하시오."))
    roll = float(input("Robot Parameter의 Rz(Roll)값을 입력하시오."))

B[0][0] = math.cos(roll)*math.cos(pitch)
B[0][1] = math.cos(roll)*math.sin(pitch)*math.sin(yaw) - math.sin(roll)*math.cos(yaw)
B[0][2] = math.cos(roll)*math.sin(pitch)*math.cos(yaw) + math.sin(roll)*math.sin(yaw)
B[1][0] = math.sin(roll)*math.cos(pitch)
B[1][1] = math.sin(roll)*math.sin(pitch)*math.sin(yaw) + math.cos(roll)*math.sin(yaw)
B[1][2] = math.sin(roll)*math.sin(pitch)*math.cos(yaw) - math.cos(roll)*math.sin(yaw)
B[2][0] = -math.sin(pitch)
B[2][1] = -math.cos(pitch)*math.sin(yaw)
B[2][2] = math.cos(pitch)*math.cos(yaw)

B[0][3] = X
B[1][3] = Y
B[2][3] = Z
B[3][3] = 1

#B = np.matrix(B)

print(B)

Base to TCP Px값을 입력하시오.1
Base to TCP Py값을 입력하시오.2
Base ti TCP Pz값을 입력하시오.3
각도는 라디안 값입니까? Y/N10
Robot Parameter의 Rx(yaw)값을 입력하시오.20
Robot Parameter의 Ry(pitch)값을 입력하시오.30
Robot Parameter의 Rz(Roll)값을 입력하시오.30
[[ 0.02379351  0.26406028 -0.96421265  1.        ]
 [-0.15240531  1.03204621  0.25754923  2.        ]
 [ 0.98803162 -0.14082313  0.06294725  3.        ]
 [ 0.          0.          0.          1.        ]]


In [13]:
#AX = XB, 정확히는 AX + XB = C와 같은 형태의 Matrix 방정식을 실베스터 방정식이라고 한다. 다만, 이는 현재, 존재하는 Method가 없어, 직접 제작
import sylvester
D = sylvester.syl(A, B)

print(D)

F = np.zeros((12, 1))

F[3] = -A[0][3]
F[7] = -A[1][3]
F[11] = -A[2][3]

print(F)

[[-0.00944985  0.15240531 -0.98803162 -0.          0.99981083  0.
   0.          0.          0.01313602  0.          0.          0.        ]
 [-0.26406028 -1.01770254  0.14082313 -0.          0.          0.99981083
   0.          0.          0.          0.01313602  0.          0.        ]
 [ 0.96421265 -0.25754923 -0.04860359 -0.98565634  0.          0.
   0.99981083  0.          0.          0.          0.01313602  0.        ]
 [-1.         -2.         -3.         -0.98565634  0.          0.
   0.          0.99981083  0.          0.          0.          0.01313602]
 [-0.9964932   0.          0.          0.         -0.010583    0.15240531
  -0.98803162 -0.          0.08262435  0.          0.          0.        ]
 [ 0.         -0.9964932   0.          0.         -0.26406028 -1.0188357
   0.14082313 -0.          0.          0.08262435  0.          0.        ]
 [ 0.          0.         -0.9964932   0.          0.96421265 -0.25754923
  -0.04973674 -0.          0.          0.          0.0826

In [11]:
X = np.linalg.solve(D, F)
print(X)

[[-0.66733568]
 [-0.05892716]
 [-1.70394956]
 [ 1.9214532 ]
 [ 0.19307213]
 [-1.54384297]
 [ 1.49241224]
 [-2.25967953]
 [ 0.00818949]
 [ 0.18380325]
 [ 0.0456661 ]
 [ 3.25721566]]


In [12]:
Result = np.zeros((4, 4))

k = 0

for i in range(4):
    for j in range(4):
        if(k <=11):
            Result[i][j] = X[k]
        k = k + 1
        
Result[3][3] = 1
print(Result)

[[-0.66733568 -0.05892716 -1.70394956  1.9214532 ]
 [ 0.19307213 -1.54384297  1.49241224 -2.25967953]
 [ 0.00818949  0.18380325  0.0456661   3.25721566]
 [ 0.          0.          0.          1.        ]]
